In [69]:
import numpy as np
import pandas as pd
from quantile_norm import quantileNormalize

In [2]:
pl_df = pd.read_csv('this_one_problem_logs_seq.csv')

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (21,23,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# read 9 experiments
problem_set = 303899
ps_df = pd.read_csv('~/git/22-Experiments/'+str(problem_set)+'_exp.csv', header=None)

In [40]:
train_df = pl_df[pl_df['user_id'].isin(ps_df[1].unique().tolist())].reset_index()

In [41]:
train_df = train_df[train_df['original'] == 1]

In [6]:
len(train_df)

944921

In [42]:
len(train_df['user_id'].unique())

928

In [43]:
sub_counts = pd.value_counts(train_df['sequence_id'])

ps_cnt = len(sub_counts[sub_counts > 400])
print ps_cnt

583


In [44]:
mask = train_df['sequence_id'].isin(sub_counts[sub_counts > 400].index)
train_df['sequence_id'][~mask] = "-"
sub_df = train_df

In [28]:
sub_df = train_df
new_df = pd.DataFrame()
new_df = pd.get_dummies(sub_df['sequence_id'])
# u'correct , u'bottom_hint, u'hint_count, u'attempt_count, u'first_response_time, first_action
# binary correctness
new_df['correct'] = np.where(sub_df['correct'] < 1, 0, 1)
new_df['bottom_hint'] = sub_df['bottom_hint']
new_df['hint_count'] = sub_df['hint_count']
new_df['attempt_count'] = sub_df['attempt_count']
new_df['first_response_time'] = sub_df['first_response_time']
# first action
new_df['first_action'] = sub_df['first_action']

norm = lambda x: (x - x.min()) / (x.max() - x.min()) if x.max() - x.min() else x/(x+1)
# normalize within problems
new_df.insert(len(new_df.columns), 'NofFRT', sub_df.groupby('sequence_id')['first_response_time'].transform(norm))
del new_df['first_response_time']

new_df.insert(len(new_df.columns), 'normalized_hint_count', sub_df.groupby('sequence_id')['hint_count'].transform(norm))
del new_df['hint_count']

new_df.insert(len(new_df.columns), 'normalized_attempt_count', sub_df.groupby('sequence_id')['attempt_count'].transform(norm))
del new_df['attempt_count']

new_df['user_id'] = sub_df['user_id']
new_df['id'] = sub_df['id']

In [33]:
start_complete_df = pd.read_csv('this_one_assignment_start_complete.csv')

In [87]:
start_complete_df = start_complete_df.drop_duplicates()

In [57]:
start_complete_df[start_complete_df['assignment_completed']==1].head()

,user_id,assignment_id,assignment_started,assignment_completed
172,52535,759956,1,1
183,52535,808385,1,1
193,78952,692338,1,1
194,78952,692343,1,1
195,78952,692347,1,1


In [37]:
sub_df['assignment_id'].head()

0    324506
1    324506
2    324506
3    324506
4    324506
Name: assignment_id, dtype: int64

In [88]:
join_df = pd.merge(sub_df, start_complete_df, on=['assignment_id', 'user_id'], how='left')

In [89]:
print len(join_df)
print len(sub_df)

944921
944921


In [58]:
type_df = pd.read_csv('this_one_assignment_type.csv')

In [59]:
type_df.head()

,sequence_id,root_sequence_id,is_rct,math_certified_items,certified_skill_builder_items,is_flat_skill_builder,textbook_items,certified_problem_solving_items,assignment_id
0,2285,2285,0,0.576923,0.0,0,0.00,0.576923,1117845
1,2427,2427,0,0.840000,0.0,0,0.84,0.080000,919829
2,8625,2634,0,0.333333,0.0,0,0.00,0.333333,486542
3,8625,2634,0,0.333333,0.0,0,0.00,0.333333,1032702
4,8625,2634,0,0.333333,0.0,0,0.00,0.333333,486542


In [83]:
type_df = type_df.drop_duplicates()

In [90]:
join2_df = pd.merge(join_df, type_df, on=['assignment_id'], how='left')

In [61]:
join2_df.head()

,index,id,assignment_id,user_id,assistment_id,problem_id,original,correct,answer_id,answer_text,...,assignment_started,assignment_completed,sequence_id_y,root_sequence_id,is_rct,math_certified_items,certified_skill_builder_items,is_flat_skill_builder,textbook_items,certified_problem_solving_items
0,29314,157998870,1105158,107178,233276,406466,1,0.0,NaN,28800,...,1,1,207868,207868,0,14.857143,11.142857,0,0.0,0.285714
1,29314,157998870,1105158,107178,233276,406466,1,0.0,NaN,28800,...,1,1,207868,207868,0,14.857143,11.142857,0,0.0,0.285714
2,29314,157998870,1105158,107178,233276,406466,1,0.0,NaN,28800,...,1,1,207868,207868,0,14.857143,11.142857,0,0.0,0.285714
3,29314,157998870,1105158,107178,233276,406466,1,0.0,NaN,28800,...,1,1,207868,207868,0,14.857143,11.142857,0,0.0,0.285714
4,29314,157998870,1105158,107178,233276,406466,1,0.0,NaN,28800,...,1,1,207868,207868,0,14.857143,11.142857,0,0.0,0.285714


In [91]:
print len(join2_df)

944921


In [62]:
pc_df = pd.read_csv('this_one_problem_correctness.csv')

In [92]:
pc_df = pc_df.drop_duplicates()

In [63]:
pc_df.head()

,problem_id,correctness,total_count
0,5,0.893976,415
1,16,0.822165,388
2,41,0.825623,562
3,52,0.711217,419
4,61,0.566910,822


In [95]:
join3_df = pd.merge(join2_df, pc_df, on=['problem_id'], how='left')

In [65]:
join3_df.head()

,index,id,assignment_id,user_id,assistment_id,problem_id,original,correct,answer_id,answer_text,...,sequence_id_y,root_sequence_id,is_rct,math_certified_items,certified_skill_builder_items,is_flat_skill_builder,textbook_items,certified_problem_solving_items,correctness,total_count
0,29314,157998870,1105158,107178,233276,406466,1,0.0,NaN,28800,...,207868,207868,0,14.857143,11.142857,0,0.0,0.285714,0.569576,1157
1,29314,157998870,1105158,107178,233276,406466,1,0.0,NaN,28800,...,207868,207868,0,14.857143,11.142857,0,0.0,0.285714,0.569576,1157
2,29314,157998870,1105158,107178,233276,406466,1,0.0,NaN,28800,...,207868,207868,0,14.857143,11.142857,0,0.0,0.285714,0.569576,1157
3,29314,157998870,1105158,107178,233276,406466,1,0.0,NaN,28800,...,207868,207868,0,14.857143,11.142857,0,0.0,0.285714,0.569576,1157
4,29314,157998870,1105158,107178,233276,406466,1,0.0,NaN,28800,...,207868,207868,0,14.857143,11.142857,0,0.0,0.285714,0.569576,1157


In [96]:
print len(join3_df)
print len(new_df)

944921
944921


In [99]:
new_df.tail()

,5897,5898,5915,5918,5920,5921,5922,5933,5945,5946,...,692193,-,correct,bottom_hint,first_action,NofFRT,normalized_hint_count,normalized_attempt_count,user_id,id
994985,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0.545388,0.0,0.022222,352112,174460390
994986,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.000073,0.0,0.044444,352112,174652383
994987,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0.000064,0.0,0.022222,352112,174652391
994988,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0.000048,0.0,0.022222,352112,174652403
994989,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0.000126,0.0,0.022222,352112,174652406


In [98]:
join3_df.tail()

,index,id,assignment_id,user_id,assistment_id,problem_id,original,correct,answer_id,answer_text,...,sequence_id_y,root_sequence_id,is_rct,math_certified_items,certified_skill_builder_items,is_flat_skill_builder,textbook_items,certified_problem_solving_items,correctness,total_count
944916,8119264,174460390,1456805,352112,87107,163443,1,1.000000,NaN,-4/3,...,10264.0,10264.0,0.0,83.0,83.0,1.0,0.0,0.0,0.697368,76.0
944917,8119265,174652383,1456805,352112,87071,163407,1,0.666667,NaN,5/8,...,10264.0,10264.0,0.0,83.0,83.0,1.0,0.0,0.0,0.734940,83.0
944918,8119266,174652391,1456805,352112,87083,163419,1,1.000000,NaN,-40/83,...,10264.0,10264.0,0.0,83.0,83.0,1.0,0.0,0.0,0.452381,84.0
944919,8119267,174652403,1456805,352112,87027,163363,1,1.000000,NaN,99/7,...,10264.0,10264.0,0.0,83.0,83.0,1.0,0.0,0.0,0.597222,72.0
944920,8119268,174652406,1456805,352112,87050,163386,1,1.000000,NaN,-19/40,...,10264.0,10264.0,0.0,83.0,83.0,1.0,0.0,0.0,0.626506,83.0


In [100]:
del new_df['user_id']
del new_df['id']
new_df['assignment_completed'] = join3_df['assignment_completed']
new_df['is_flat_skill_builder'] = join3_df['is_flat_skill_builder']
new_df['textbook_items'] = join3_df['textbook_items']
new_df['certified_problem_solving_items'] = join3_df['certified_problem_solving_items']
new_df['correctness'] = join3_df['correctness']
new_df['user_id'] = join3_df['user_id']
new_df['id'] = join3_df['id']

In [102]:
new_df['correctness'].fillna(0.5, inplace=True)

In [109]:
new_df['assignment_completed'].fillna(0, inplace=True)
new_df['is_flat_skill_builder'].fillna(0, inplace=True)
new_df['textbook_items'].fillna(0, inplace=True)
new_df['certified_problem_solving_items'].fillna(0, inplace=True)

In [111]:
new_df.dropna(inplace=True)

In [112]:
for col in new_df.columns:
    if new_df[col].isnull().values.any():
        print col

In [103]:
new_df.isnull().values.any()

True

In [31]:
with open(str(problem_set)+'_ps_list', 'w') as file:
    for ite in new_df.columns[:ps_cnt].tolist():
        file.write(str(ite) + "\n")

In [113]:
new_df.to_csv(str(problem_set)+'_extra_pl_train_data.csv', index=False)